## Imports

In [59]:
import pandas as pd
import numpy as np
from pathlib import Path

## Data setup

In [67]:
data_dir = Path('data')
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet*')
)
df

,BILL_DATE,MATERIAL,SALES_DIST,SOLD_TO,SHIP_TO,PIEZAS
8511,2019-03-03,2641,100482,6001932,7003984,1
8512,2019-03-03,12916,100482,6001932,7003984,1
8513,2019-03-03,2464,100482,6001932,7003984,1
9478,2019-03-03,11074,100482,6001932,7003984,1
9479,2019-03-03,20306,100482,6001932,7003984,1
...,...,...,...,...,...,...
187373097,2021-03-31,2296,104862,6003121,7036861,1
187373098,2021-03-31,3590,104862,6003121,7036861,1
187373099,2021-03-31,8505,104862,6003121,7036861,1
187373100,2021-03-31,16951,104862,6003121,7036861,1


## Top 10 most sold products

In [69]:
total_sales = full_df.groupby(['MATERIAL'])['PIEZAS'].sum()
total_sales = total_sales.sort_values(ascending=False)
total_sales.head(10)

MATERIAL
1564     600
4373     379
3444     240
21256    222
6969     210
4103     207
15590    205
4101     200
42830    191
12091    184
Name: PIEZAS, dtype: int64

## Weekly sales for most sold product

In [70]:
prod_1 = full_df[full_df['MATERIAL'] == 1564]
prod_1['Week'] = pd.to_datetime(prod_1['BILL_DATE']) - pd.to_timedelta(7, unit='d')
prod_1 = prod_1.filter(['Week', 'PIEZAS']).groupby([pd.Grouper(key='Week', freq='W-MON')]).sum().reset_index()
prod_1

<ipython-input-70-1ebb41787e88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_1['Week'] = pd.to_datetime(prod_1['BILL_DATE']) - pd.to_timedelta(7, unit='d')


,Week,PIEZAS
0,2019-03-18,100
1,2019-03-25,0
2,2019-04-01,0
3,2019-04-08,0
4,2019-04-15,0
...,...,...
65,2020-06-15,0
66,2020-06-22,0
67,2020-06-29,0
68,2020-07-06,0
